In [1]:
%cd /content/drive/MyDrive/Colab/2.1.Text_Classification

/content/drive/MyDrive/Colab/2.1.Text_Classification


In [2]:
%pwd

'/content/drive/MyDrive/Colab/2.1.Text_Classification'

# 1. Tải bộ dữ liệu:

In [4]:
# https://drive.google.com/file/d/1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R/view?usp=sharing
!gdown --id 1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R
To: /content/drive/MyDrive/Colab/2.1.Text_Classification/2cls_spam_text_cls.csv
100% 486k/486k [00:00<00:00, 33.9MB/s]


# 2. Import các thư viện cần thiết

In [3]:
import string  # Cung cấp các hàm cơ bản để thao tác với chuỗi ký tự.

import nltk  # (Natural Language Toolkit):thư viện xử lý ngôn ngữ tự nhiên
nltk.download('stopwords')
nltk.download('punkt')

import pandas as pd # Cung cấp các cấu trúc dữ liệu hiệu quả và các công cụ để làm việc với dữ liệu
import numpy as np  # Cung cấp các đối tượng mảng đa chiều và các hàm toán học để làm việc với các mảng này
import matplotlib.pyplot as plt #

# Thư viện ML phổ biến, giúp xây dựng và triển khai các mô hình ML phức tạp một cách nhanh chóng.
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 3. Đọc dữ liệu

In [4]:
DATASET_PATH = '/content/drive/MyDrive/Colab/2.1.Text_Classification/2cls_spam_text_cls.csv'
df = pd.read_csv(DATASET_PATH)

messages = df['Message'].values.tolist()
labels = df['Category'].values.tolist()

In [5]:
# review dataset
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


# 4. Tiền xử lý dữ liệu

## Xử lý dữ liệu đặc trưng

In [6]:
# chuyển tất cả chuỗi sang chữ thường
def lowercase(text):
    return text.lower()

# xóa dấu câu
def punctuation_removal(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# tách từ
def tokenize(text):
    return nltk.word_tokenize(text)

# xóa stopwords
def remove_stopwords(tokens):
    stop_words = nltk.corpus.stopwords.words('english')
    return [token for token in tokens if token not in stop_words]

# đưa từ về dạng gốc
def stemming(tokens):
    stemmer = nltk.PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

# tiền xử lý văn bản (tổng hợp)
def preprocess_text(text):
    text = lowercase(text)
    text = punctuation_removal(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)
    return tokens

In [9]:
# tạo một bộ từ điển (Dictionary), chứa tất các từ hoặc ký tự xuất hiện trong toàn bộ Messages sau khi tiền xử lý.
def create_dictionary(messages):
    dictionary = []
    for tokens in messages:
        for token in tokens:
            if token not in dictionary:
                dictionary.append(token)
    return dictionary

# đếm tần xuất các từ trong bộ từ điển xuất hiện trong toàn bộ messages (vector đặc trưng cho các message)
def create_features(tokens, dictionary):
    features = np.zeros(len(dictionary))
    for token in tokens:
        if token in dictionary:
            features[dictionary.index(token)] += 1
    return features

In [8]:
# tiền xử lý toàn bộ nội dung message(trong dataset)
messages = [preprocess_text(message) for message in messages]

In [10]:
# tạo bộ từ điển từ messages
dictionary = create_dictionary(messages)

# tạo vector đặc trưng cho các message
X = np.array([create_features(tokens, dictionary) for tokens in messages])

## Xử lý dữ liệu nhãn

In [11]:
le = LabelEncoder()
y = le.fit_transform(labels)
print(f'Classes: {le.classes_}')
print(f'Encoded labels: {y}')

Classes: ['ham' 'spam']
Encoded labels: [0 0 1 ... 0 0 0]


# 5. Chia bộ dữ liệu train/val/test

In [12]:
VAL_SIZE = 0.2 # 20% của bộ dữ liệu gốc ~ 20%
TEST_SIZE = 0.125 # 12.5% của 80%(phần còn lại của bộ dữ liệu gốc) ~ 10%
# phần còn lại thuộc về TRAIN_SIZE ~ 70% của bộ dữ liệu gốc
SEED = 0

X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=VAL_SIZE,
                                                  shuffle=True,
                                                  random_state=SEED)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,
                                                    test_size=TEST_SIZE,
                                                    shuffle=True,
                                                    random_state=SEED)

# 6. Huấn luyện mô hình:

In [14]:
%%time
model = GaussianNB()

print('Start training...')
model = model.fit(X_train, y_train)
print('Training completed!')

Start training...
Training completed!
CPU times: user 424 ms, sys: 194 ms, total: 618 ms
Wall time: 675 ms


# 7. Đánh giá mô hình

In [15]:
# đánh giá lại model đã train ở Step6
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f'Val accuracy: {val_accuracy}')
print(f'Test accuracy: {test_accuracy}')

Val accuracy: 0.8816143497757848
Test accuracy: 0.8602150537634409


# 8. Thực hiện dự đoán

In [17]:
# dùng model đã train ở step6 dự đoán 1 message được truyền vào
def predict(text, model, dictionary):
    processed_text = preprocess_text(text)
    features = create_features(text, dictionary)
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    prediction_cls = le.inverse_transform(prediction)[0]

    return prediction_cls

In [18]:
# thử nghiệm dự đoán
test_input = 'I am actually thinking a way of doing something useful'
prediction_cls = predict(test_input, model, dictionary)
print(f'Prediction: {prediction_cls}')

Prediction: ham
